In [1]:
import pandas as pd
import numpy as np

In [3]:
local_data_folder = "/Users/colleendriscoll/Dropbox/Classes/CS 109A/CS 109A Final project/data/"

## Correlation table with p-values

In [2]:
from scipy.stats import pearsonr

In [4]:
comb_data = pd.read_csv(local_data_folder + "combined_data.csv")
comb_data.head()

,year,dist_id,dem_vote_share,congress,icpsr,nominate_dim1,nominate_dim2,nokken_poole_dim1,nokken_poole_dim2,age,...,senate,median_HH_income,median_age,mortgage_cost,unemp_rate,bachelor_deg_perc,black_perc,HS_diploma_perc,white_perc,redistricted
0,2006,AK_1,0.414254,109.0,14066.0,0.279,0.014,0.367,-0.047,73.0,...,0.484848,25793.0,33.5,1015.0,9.4,0.107839,0.032051,0.177972,0.686767,0.0
1,2008,AK_1,0.472837,110.0,14066.0,0.279,0.014,0.291,0.269,75.0,...,0.500000,30271.0,33.0,1166.0,7.7,0.108886,0.036257,0.164006,0.691129,0.0
2,2010,AK_1,0.306755,111.0,14066.0,0.279,0.014,0.242,0.642,77.0,...,0.581633,28202.0,33.8,1167.0,9.6,0.116734,0.034610,0.159744,0.674308,0.0
3,2012,AK_1,0.309172,112.0,14066.0,0.279,0.014,0.338,-0.052,79.0,...,0.520408,29932.0,33.8,1239.0,7.8,0.108769,0.035829,0.178108,0.665188,0.0
4,2014,AK_1,0.445620,113.0,14066.0,0.279,0.014,0.265,0.116,81.0,...,0.540816,31557.0,33.3,1258.0,7.6,0.116235,0.033649,0.179152,0.656391,0.0


In [5]:
corr_table_cols_to_drop = ['dist_id', 'congress', 'icpsr',
                          'nominate_dim1', 'nominate_dim2']
corr_table_data = comb_data.drop(corr_table_cols_to_drop,
                                axis = 1, inplace = False)
## Reordering columns
corr_table_data = corr_table_data[['year', 'dem_vote_share',
                                   'dem_prior_vote_share', 
                                   'dem_incumbent',
                                   'redistricted',
                                   'president', 'house',
                                   'senate', 'nokken_poole_dim1',
                                   'nokken_poole_dim2', 'age',
                                   'white_perc', 'black_perc',
                                   'HS_diploma_perc',
                                   'bachelor_deg_perc',
                                   'unemp_rate', 'mortgage_cost',
                                   'median_HH_income', 'median_age']]

In [6]:
def calculate_pvalues(df):
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(pearsonr(df[r], df[c])[1], 4)
    return pvalues

In [7]:
rho = corr_table_data.corr()
rho = rho.round(2)
p_values = calculate_pvalues(corr_table_data)
# create three masks
r1 = rho.applymap(lambda x: '{}*'.format(x))
r2 = rho.applymap(lambda x: '{}**'.format(x))
r3 = rho.applymap(lambda x: '{}***'.format(x))
# apply them where appropriate
rho = rho.mask(p_values<=0.1,r1)
rho = rho.mask(p_values<=0.05,r2)
rho = rho.mask(p_values<=0.01,r3)

rho

,year,dem_vote_share,dem_prior_vote_share,dem_incumbent,redistricted,president,house,senate,nokken_poole_dim1,nokken_poole_dim2,age,white_perc,black_perc,HS_diploma_perc,bachelor_deg_perc,unemp_rate,mortgage_cost,median_HH_income,median_age
year,1.0***,-0.04**,-0.06***,-0.06***,-0.01,0.29***,-0.08***,-0.32***,0.07***,-0.01,0.06***,-0.03*,-0,-0.08***,0.18***,-0.2***,0.13***,0.28***,0.18***
dem_vote_share,-0.04**,1.0***,0.77***,0.73***,-0.02,-0.15***,0.01,-0.1***,-0.75***,-0.19***,0.15***,-0.55***,0.35***,-0.22***,0.07***,0.19***,0.26***,0.03*,-0.21***
dem_prior_vote_share,-0.06***,0.77***,1.0***,0.78***,-0.08***,-0.03,0.12***,0.08***,-0.79***,-0.16***,0.15***,-0.52***,0.33***,-0.18***,0.03,0.29***,0.23***,-0,-0.19***
dem_incumbent,-0.06***,0.73***,0.78***,1.0***,-0.05***,0,0.08***,0.08***,-0.94***,-0.09***,0.16***,-0.45***,0.27***,-0.18***,0.03,0.25***,0.2***,-0,-0.18***
redistricted,-0.01,-0.02,-0.08***,-0.05***,1.0***,0.27***,-0.38***,0.06***,0.05***,0.02,0,-0.01,0.03,-0.02,-0.01,0.23***,-0.03*,-0.05***,-0.01
president,0.29***,-0.15***,-0.03,0,0.27***,1.0***,-0.31***,0.45***,0.02,-0.01,0.01,-0,0,-0.04**,0.04**,0.38***,0.02,-0.01,0.07***
house,-0.08***,0.01,0.12***,0.08***,-0.38***,-0.31***,1.0***,0.43***,-0.07***,0.03*,0.02,0.01,0,0,-0.02,0.18***,0.02,-0.01,-0.02
senate,-0.32***,-0.1***,0.08***,0.08***,0.06***,0.45***,0.43***,1.0***,-0.06***,0.02,-0.01,0.02,0,0.01,-0.08***,0.56***,-0.04**,-0.17***,-0.04**
nokken_poole_dim1,0.07***,-0.75***,-0.79***,-0.94***,0.05***,0.02,-0.07***,-0.06***,1.0***,0.11***,-0.21***,0.48***,-0.29***,0.18***,-0.04**,-0.26***,-0.23***,-0.01,0.18***
nokken_poole_dim2,-0.01,-0.19***,-0.16***,-0.09***,0.02,-0.01,0.03*,0.02,0.11***,1.0***,-0.06***,0.13***,0.03*,0.29***,-0.35***,0.02,-0.38***,-0.29***,-0.08***
